In [ ]:
import pandas as pd
import numpy as np
import time
import os
import joblib
from datetime import datetime, timezone
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from alpaca_trade_api.rest import REST

# --- CONFIG ---
API_KEY = 'YOUR_ALPACA_API_KEY'
SECRET_KEY = 'YOUR_ALPACA_SECRET_KEY'
BASE_URL = 'https://paper-api.alpaca.markets'

SYMBOLS = ['AAPL', 'MSFT', 'TSLA', 'NVDA', 'GOOG']
TRADE_QTY = 1
STOP_LOSS_PCT = 0.10
LOG_FILE = 'trade_log.csv'
MODEL_FILE = 'xgb_model.joblib'

api = REST(API_KEY, SECRET_KEY, BASE_URL)

# --- Load CSV data ---
def load_symbol_data(symbol):
    filename = f'{symbol}_5min.csv'
    df = pd.read_csv(filename, parse_dates=['Datetime'])
    df.rename(columns={'Datetime': 'timestamp'}, inplace=True)
    df['symbol'] = symbol
    df.sort_values('timestamp', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# --- Features ---
def add_features(df):
    df = df.copy()
    df['return'] = df['Close'].pct_change()
    df['volatility'] = df['return'].rolling(window=5).std()
    df['ma'] = df['Close'].rolling(window=5).mean()
    df['ma_diff'] = df['Close'] - df['ma']
    df['target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df.dropna(inplace=True)
    return df

# --- Get open position ---
def get_position(symbol):
    try:
        pos = api.get_position(symbol)
        return int(pos.qty), float(pos.avg_entry_price)
    except:
        return 0, 0.0

# --- Log trade ---
def log_trade(symbol, action, price):
    now = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
    try:
        account = api.get_account()
        equity = float(account.equity)
        cash = float(account.cash)
    except:
        equity, cash = 0, 0
    row = {
        'time': now,
        'symbol': symbol,
        'action': action,
        'price': price,
        'equity': equity,
        'cash': cash
    }
    df = pd.DataFrame([row])
    if os.path.exists(LOG_FILE):
        df.to_csv(LOG_FILE, mode='a', header=False, index=False)
    else:
        df.to_csv(LOG_FILE, index=False)

# --- Trade logic ---
def trade(symbol, model, latest_features, price):
    qty, avg_price = get_position(symbol)

    if qty > 0 and price < avg_price * (1 - STOP_LOSS_PCT):
        try:
            api.submit_order(symbol=symbol, qty=qty, side='sell', type='market', time_in_force='gtc')
            print(f"STOP-LOSS SELL: {symbol} @ {price:.2f}")
            log_trade(symbol, 'stop-loss', price)
            return
        except Exception as e:
            print(f"Stop-loss error for {symbol}: {e}")
            return

    prediction = model.predict(latest_features)[0]

    if prediction == 1 and qty == 0:
        try:
            api.submit_order(symbol=symbol, qty=TRADE_QTY, side='buy', type='market', time_in_force='gtc')
            print(f"BUY: {symbol} @ {price:.2f}")
            log_trade(symbol, 'buy', price)
        except Exception as e:
            print(f"Buy error for {symbol}: {e}")
    elif prediction == 0 and qty > 0:
        try:
            api.submit_order(symbol=symbol, qty=qty, side='sell', type='market', time_in_force='gtc')
            print(f"SELL: {symbol} @ {price:.2f}")
            log_trade(symbol, 'sell', price)
        except Exception as e:
            print(f"Sell error for {symbol}: {e}")
    else:
        print(f"HOLD: {symbol} @ {price:.2f}")

# --- Train the model ---
def train_model(all_data):
    df = add_features(all_data)
    X = df[['return', 'volatility', 'ma_diff']]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    acc = model.score(X_test, y_test)
    print(f"Model trained. Accuracy: {acc*100:.2f}%")
    joblib.dump(model, MODEL_FILE)
    return model

# --- Main Loop ---
def main():
    print("Starting AI paper trading bot (replay mode)...")

    # Load data
    data = {symbol: load_symbol_data(symbol) for symbol in SYMBOLS}
    all_data = pd.concat(data.values(), ignore_index=True)
    model = train_model(all_data)

    indexes = {symbol: 0 for symbol in SYMBOLS}
    max_len = max(len(df) for df in data.values())

    for step in range(max_len):
        for symbol in SYMBOLS:
            df = data[symbol]
            idx = indexes[symbol]
            if idx >= len(df):
                continue

            current_df = df.iloc[:idx+1]
            features_df = add_features(current_df)

            if features_df.empty:
                indexes[symbol] += 1
                continue

            latest_features = features_df[['return', 'volatility', 'ma_diff']].iloc[[-1]]
            price = features_df['Close'].iloc[-1]
            trade(symbol, model, latest_features, price)
            indexes[symbol] += 1

        print(f"Step {step+1}/{max_len} complete. Sleeping 1 sec...\n")
        time.sleep(1)

    print("Replay complete. Bot stopped.")

if __name__ == '__main__':
    main()
